In [2]:
import sys
!{sys.executable} -m ensurepip --upgrade

Looking in links: c:\Users\Playdata\AppData\Local\Temp\tmp9bl38tbz
Processing c:\users\playdata\appdata\local\temp\tmp9bl38tbz\setuptools-65.5.0-py3-none-any.whl
Processing c:\users\playdata\appdata\local\temp\tmp9bl38tbz\pip-24.0-py3-none-any.whl


In [1]:
%pip install pandas psycopg2-binary python-dotenv matplotlib seaborn

  Using cached pandas-2.2.3-cp311-cp311-win_amd64.whl.metadata (19 kB)
  Using cached python_dotenv-1.1.0-py3-none-any.whl.metadata (24 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ---------------------------------------- 60.8/60.8 kB 3.2 MB/s eta 0:00:00
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
     ---------------------------------------- 0.0/108.2 kB ? eta -:--:--
     -------------------------------------- 108.2/108.2 kB 6.1 MB/s eta 0:00:00
  Using cached kiwisolver-1.4.8-cp311-cp311-win_amd64.whl.metadata (6.3 kB)
  Using cached pillow-11.2.1-cp311-cp311-win_amd64.whl.metadata (9.1 kB)
  Using cached pyparsing-3.2.3-py3-none-any.whl.metadata (5.0 kB)
Using cached pandas-2.2.3-cp311-cp311-win_amd64.whl (11.6 MB)
   -------------


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import pandas as pd
import psycopg2 # PostgreSQL 어댑터
from dotenv import load_dotenv
import matplotlib.pyplot as plt
import seaborn as sns # 좀 더 보기 좋은 시각화를 위해 (선택 사항)

# --- 환경 변수 로드 ---
# .env 파일 경로를 필요에 따라 수정하세요.
# 옵션 1: 노트북 파일과 동일한 디렉토리 (SKN10-FINAL-1Team)에 .env 파일이 있는 경우
dotenv_path_option1 = '.env'
# 옵션 2: my_langraph_agent 디렉토리에 .env 파일이 있는 경우 (SKN10-FINAL-1Team 기준 상대 경로)
dotenv_path_option2 = os.path.join('my_langraph_agent', '.env')

# my_langraph_agent 디렉토리에서 먼저 .env 파일을 찾고, 없으면 현재 디렉토리에서 찾습니다.
if os.path.exists(dotenv_path_option2):
    load_dotenv(dotenv_path=dotenv_path_option2)
    print(f".env 파일을 다음 경로에서 로드했습니다: {os.path.abspath(dotenv_path_option2)}")
elif os.path.exists(dotenv_path_option1):
    load_dotenv(dotenv_path=dotenv_path_option1)
    print(f".env 파일을 다음 경로에서 로드했습니다: {os.path.abspath(dotenv_path_option1)}")
else:
    print("경고: .env 파일을 찾을 수 없습니다. DB 연결 정보가 환경 변수로 설정되었거나 .env 파일 경로를 확인해주세요.")

# --- 데이터베이스 연결 정보 ---
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT", "5432") # 설정되지 않은 경우 기본값 5432
DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")

# 모든 필수 정보가 로드되었는지 확인
if not all([DB_HOST, DB_NAME, DB_USER, DB_PASSWORD]):
    missing = [var_name for var_name, var_val in [("DB_HOST", DB_HOST), ("DB_NAME", DB_NAME), ("DB_USER", DB_USER), ("DB_PASSWORD", DB_PASSWORD)] if not var_val]
    print(f"오류: PostgreSQL 연결 정보가 누락되었습니다: {', '.join(missing)}. .env 파일 또는 환경 변수를 확인해주세요.")
else:
    print("데이터베이스 연결 정보를 성공적으로 로드했습니다.")
    print(f"연결 대상: {DB_NAME}@{DB_HOST}:{DB_PORT}")


.env 파일을 다음 경로에서 로드했습니다: c:\development\github\SKN10-FINAL-1Team\my_langraph_agent\.env
데이터베이스 연결 정보를 성공적으로 로드했습니다.
연결 대상: mydatabase@35.170.244.126:5432


In [5]:
# 새 셀에서 실행
import os, pandas as pd, psycopg2
from dotenv import load_dotenv
# (셀 1의 .env 로드 및 DB 접속 정보 설정 코드 필요)

if all([DB_HOST, DB_NAME, DB_USER, DB_PASSWORD]):
    try:
        conn_check = psycopg2.connect(host=DB_HOST, database=DB_NAME, user=DB_USER, password=DB_PASSWORD, port=DB_PORT)
        query_tables = "SELECT table_name FROM information_schema.tables WHERE table_schema = 'public';"
        tables_df = pd.read_sql_query(query_tables, conn_check)
        print("데이터베이스의 테이블 목록:")
        print(tables_df)
        if conn_check:
            conn_check.close()
    except Exception as e:
        print(f"테이블 목록 조회 중 오류: {e}")
else:
    print("DB 연결 정보 부족")

C:\Users\Playdata\AppData\Local\Temp\ipykernel_11288\3996009675.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tables_df = pd.read_sql_query(query_tables, conn_check)


데이터베이스의 테이블 목록:
                       table_name
0         pg_stat_statements_info
1              pg_stat_statements
2                  django_session
3                   news_articles
4               django_migrations
5             django_content_type
6                 auth_permission
7                      auth_group
8          auth_group_permissions
9                   organizations
10                          users
11          summary_news_keywords
12                   users_groups
13         users_user_permissions
14                      llm_calls
15               django_admin_log
16                  chat_sessions
17                  chat_messages
18              telecom_customers
19  knowledge_summarynewskeywords
20          checkpoint_migrations
21                    checkpoints
22               checkpoint_blobs
23              checkpoint_writes
24               git_repositories
25                     code_files
26                      documents
27                   embed_chunk

In [6]:
# --- PostgreSQL 연결 및 데이터 조회 ---
# 실제 실행할 SQL 쿼리로 변경하세요.
sql_query = "SELECT * FROM summary_news_keywords LIMIT 10;" # 예시 쿼리

conn = None
df = pd.DataFrame() # 빈 DataFrame으로 초기화

if all([DB_HOST, DB_NAME, DB_USER, DB_PASSWORD]):
    try:
        print(f"PostgreSQL 연결 시도 중: {DB_USER}@{DB_HOST}:{DB_PORT}/{DB_NAME}")
        conn = psycopg2.connect(
            host=DB_HOST,
            database=DB_NAME,
            user=DB_USER,
            password=DB_PASSWORD,
            port=DB_PORT
        )
        print("PostgreSQL에 성공적으로 연결되었습니다.")
        
        print(f"SQL 쿼리 실행 중: {sql_query}")
        df = pd.read_sql_query(sql_query, conn)
        
        if df.empty:
            print("SQL 쿼리 결과 데이터가 없습니다.")
        else:
            print("데이터를 성공적으로 조회했습니다:")
            print(df.head()) # 조회된 데이터의 첫 5행 출력

    except psycopg2.Error as e:
        print(f"PostgreSQL 연결 또는 쿼리 실행 중 오류 발생: {e}")
    finally:
        if conn:
            conn.close()
            print("PostgreSQL 연결이 닫혔습니다.")
else:
    print("연결 정보 부족으로 데이터베이스에 연결할 수 없습니다.")

PostgreSQL 연결 시도 중: myuser@35.170.244.126:5432/mydatabase
PostgreSQL에 성공적으로 연결되었습니다.
SQL 쿼리 실행 중: SELECT * FROM summary_news_keywords LIMIT 10;


C:\Users\Playdata\AppData\Local\Temp\ipykernel_11288\1152581256.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql_query, conn)


데이터를 성공적으로 조회했습니다:
                                     id        date                   keyword  \
0  3a89a409-6e0f-4c36-8003-023caeb30047  2025-05-29                      서비스를   
1  b9a65681-a325-43a0-86f8-7ce60b2922d6  2025-05-29                       SKT   
2  8b163d2b-0a74-4fac-b7f2-e295bad3af97  2025-05-29  SKT,ssss,ssssdds,sdasdas   
3  34c1875f-b144-4b0b-a707-b29280f35573  2025-05-29                       SKT   
4  0c77bb54-ada3-4fe8-9e44-e504bbf281ee  2025-05-29                     SK텔레콤   

                                  title  \
0        이채연, 정치색 논란 차단‥센스 폭발한 사전투표 인증샷   
1  [DD퇴근길] 페북 유명인 사칭광고 사라질까…안면인식 도입 초읽기   
2                  대선후보의 SKT 해킹사고 대응공약은   
3  SKT 유심 교체자 500만 명 돌파…"6월 초 모두 교체 예정"   
4            양구군농협, ‘찾아가는 유심 교체 서비스’ 진행   

                                             summary  \
0  이채연, 정치색 논란 차단‥센스 폭발한 사전투표 인증샷입력공감좋아요응원해요축하해요기...   
1  29일 업계에 따르면 SK하이닉스와 SK에코플랜트는 미국 현지 인디애나주에 팀을 파...   
2  국민의힘·민주당 '사이버 위협 공약' 분석 대형사고 계기로 보안정책강화 '한목소리'...   
3  SK텔레콤 누적